In [1]:
import sys
import subprocess
import cdsapi
import os
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
from matplotlib import pyplot as plt

import errno
from dask.diagnostics import ProgressBar
import scipy
import netCDF4
from dask.distributed import progress

In [2]:
from dask.distributed import Client

client = Client('tcp://127.0.0.1:35461')
client

Client Scheduler: tcp://127.0.0.1:35461 Dashboard: http://127.0.0.1:49333/status,Cluster Workers: 56 Cores: 56 Memory: 270.36 GB


In [4]:
mychunk = {'latitude':25,'longitude':25,'time': -1}
def RunRHExtremes(zone):
    zone = "SAR"
    basedir = '/mnt/DataDrive2/data/mjolly/era5-land'
    #basedir = '/mnt/nas/jolly'
    yrs = range(2015,2021)
    for y in yrs:
        print("Summarizing Temperatures for year:",y)
        ifile = '%s/era5-land-%s-2m_temperature-%s-*nc' % (basedir,zone,y)
        print("Outfile: %s:" % (ifile))
        zone_rh = xr.open_mfdataset(ifile,chunks=mychunk,parallel=True,engine='netcdf4')
        outds = zone_rh.groupby('time.dayofyear').min('time')
        ifile = '%s/era5-land-%s-2m_tmin-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )
        outds = zone_rh.groupby('time.dayofyear').max('time')
        ifile = '%s/era5-land-%s-2m_tmax-%s.nc' % (basedir,zone,y)
        outds.to_netcdf(ifile,format= 'NETCDF4' )

%time RunRHExtremes("SAR")

Summarizing Temperatures for year: 2015
Outfile: /mnt/DataDrive2/data/mjolly/era5-land/era5-land-SAR-2m_temperature-2015-*nc:


KeyboardInterrupt: 